In [1]:
from collections import defaultdict
import copy
import matplotlib.pyplot as plt
from circuit.parser import get_circuit_duration
from plot.plot import plot_duration_fidelity, plot_top_ratio, find_error_path
import random
import numpy as np

from circuit import gen_random_circuits, label_ground_truth_fidelity
from upstream import RandomwalkModel
from downstream import FidelityModel
from simulator import NoiseSimulator
from utils.backend import devide_chip, gen_grid_topology, get_grid_neighbor_info, Backend, topology_to_coupling_map
from utils.backend import default_basis_single_gates, default_basis_two_gates
import pickle

In [15]:
with open(f"execute_18bit/split_dataset_execute_18bits_train_0_2500.pkl","rb")as f:
    train_dataset, test_dataset = pickle.load(f)

In [11]:
cir = copy.deepcopy(train_dataset[0])

In [12]:
cir['layer2gates']

[[{'name': 'ry', 'qubits': [0], 'params': [4.39822971502571], 'id': 10},
  {'name': 'rx', 'qubits': [1], 'params': [0.3141592653589793], 'id': 11},
  {'name': 'ry', 'qubits': [2], 'params': [3.7699111843077517], 'id': 12},
  {'name': 'ry', 'qubits': [3], 'params': [0.6283185307179586], 'id': 13},
  {'name': 'ry', 'qubits': [4], 'params': [2.5132741228718345], 'id': 14}],
 [{'name': 'ry', 'qubits': [4], 'params': [2.5132741228718345], 'id': 26},
  {'name': 'rz', 'qubits': [3], 'params': [0.3141592653589793], 'id': 27},
  {'name': 'cz', 'qubits': [0, 2], 'params': [], 'id': 28}],
 [{'name': 'rz', 'qubits': [4], 'params': [3.7699111843077517], 'id': 39},
  {'name': 'ry', 'qubits': [3], 'params': [5.026548245743669], 'id': 40},
  {'name': 'cz', 'qubits': [1, 2], 'params': [], 'id': 41},
  {'name': 'rx', 'qubits': [0], 'params': [5.026548245743669], 'id': 42}],
 [{'name': 'rz', 'qubits': [3], 'params': [1.2566370614359172], 'id': 52},
  {'name': 'rx', 'qubits': [2], 'params': [1.25663706143

In [17]:
for cir in test_dataset:
    for idx, gate in enumerate(cir['gates']):
        gate['id'] = idx
        
    layer2gates = []
    idx = 0
    for layer, gates in enumerate(cir['layer2gates']):
        new_layer = []
        for _ in range(len(cir['layer2gates'][layer])):
            cir['gate2layer'][idx] = layer
            new_layer.append(cir['gates'][idx])
            idx += 1
        layer2gates.append(new_layer)
    cir['layer2gates'] = layer2gates

In [18]:
train_dataset[0]

{'qiskit_circuit': None,
 'num_qubits': 5,
 'divide_qubits': [[5, 10, 11, 16, 17]],
 'gate_paths': [{'ry,5', 'ry,5-parallel-rx,4', 'ry,5-parallel-ry,11'},
  {'rx,10',
   'rx,10-parallel-rx,4',
   'rx,10-parallel-ry,11',
   'rx,10-parallel-ry,16',
   'rx,10-parallel-ry,9'},
  {'ry,11',
   'ry,11-parallel-rx,10',
   'ry,11-parallel-ry,17',
   'ry,11-parallel-ry,5'},
  {'ry,16',
   'ry,16-parallel-rx,10',
   'ry,16-parallel-rx,15',
   'ry,16-parallel-ry,17'},
  {'ry,17', 'ry,17-parallel-ry,11', 'ry,17-parallel-ry,16'},
  {'ry,17',
   'ry,17-former-ry,11',
   'ry,17-former-ry,16',
   'ry,17-former-ry,17',
   'ry,17-parallel-cz,5,11',
   'ry,17-parallel-rz,16'},
  {'rz,16',
   'rz,16-former-rx,10',
   'rz,16-former-rx,15',
   'rz,16-former-ry,16',
   'rz,16-former-ry,17',
   'rz,16-parallel-ry,15',
   'rz,16-parallel-ry,17'},
  {'cz,5,11',
   'cz,5,11-former-rx,10',
   'cz,5,11-former-rx,4',
   'cz,5,11-former-ry,11',
   'cz,5,11-former-ry,17',
   'cz,5,11-former-ry,5',
   'cz,5,11-parallel

In [ ]:
with open(f"execute_18bit/split_dataset_execute_18bits_train_0_2500.pkl","wb")as f:
    pickle.dump((train_dataset, test_dataset),f)